This notebook is pre trained model for creating chatbot with good accuracy for this dataset. However, Accuracy range will be different.

You can add dataset with .csv file. But, you need to transfer your data json file into csv file format using python script. I tried it but don't seem to have converted successfully.

## **Import and load the data file**

We import the necessary packages for our chatbot and initialize the variables we will use in our Python project.
The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [1]:
import nltk
nltk.download('punkt')#Sentence tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gicha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# **Preprocessing**

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() # read json file
intents = json.loads(data_file) # load json file

When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Based on the requirements we need to apply various operations to preprocess the 
data.
- Tokenizing is the most basic and first thing you can do on text data. 
- Tokenizing is the process of breaking the whole text into small parts like words.
- Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gicha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gicha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Now we will lemmatize each word and remove duplicate words from the list. 
- Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [8]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

448 documents
 [(['When', 'did', 'Iserve', 'Africa', 'begin', '?'], 'begin'), (['Could', 'you', 'share', 'your', 'testimony', '?'], 'Testimony_experiences_stories'), (['What', "'s", 'your', 'personal', 'journey', 'with', 'faith', '?'], 'Testimony_experiences_stories'), (['Do', 'you', 'have', 'a', 'personal', 'testimony', 'to', 'share', '?'], 'Testimony_experiences_stories'), (['How', 'did', 'iServe', 'Africa', 'start', '?'], 'History'), (['What', 'is', 'the', 'history', 'behind', 'iServe', 'Africa', '?'], 'History'), (['Can', 'you', 'tell', 'me', 'about', 'the', 'beginnings', 'of', 'iServe', 'Africa', '?'], 'History'), (['How', 'can', 'I', 'get', 'in', 'touch', 'with', 'iServe', 'Africa', '?'], 'Contact_email'), (['What', 'are', 'the', 'contact', 'details', 'for', 'iServe', 'Africa', '?'], 'Contact_email'), (['Can', 'you', 'provide', 'me', 'with', 'the', 'contact', 'information', 'of', 'iServe', 'Africa', '?'], 'Contact_email'), (['about', 'iServe', 'Africa'], 'about'), (['background',

# **Training Model**

Now, we will create the training data in which we will provide the input and the output. 
- Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers

In [9]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

# **Build the model** 

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5'.

In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])


First layer: [[ 0.08274695  0.05851296  0.08639891 ...  0.07655198 -0.04375339
  -0.06947809]
 [-0.10285683  0.10568642  0.0158431  ... -0.05827478 -0.10616069
  -0.06305213]
 [ 0.07818552  0.06454479 -0.09565143 ... -0.10787142 -0.04560909
   0.07865577]
 ...
 [ 0.01430487 -0.02923286  0.10825484 ... -0.04950082 -0.07278948
   0.10079341]
 [-0.02733597  0.03685733  0.05918891 ...  0.05796736  0.05342748
  -0.08087586]
 [ 0.07793497  0.03087769 -0.08346767 ... -0.07847364  0.04723107
  -0.08878833]]


In [12]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200


90/90 [==============================] - 2s 3ms/step - loss: 4.6726 - accuracy: 0.0134
Epoch 2/200
90/90 [==============================] - 0s 3ms/step - loss: 4.5791 - accuracy: 0.0848
Epoch 3/200
90/90 [==============================] - 0s 3ms/step - loss: 4.3761 - accuracy: 0.1138
Epoch 4/200
90/90 [==============================] - 0s 3ms/step - loss: 4.1639 - accuracy: 0.1094
Epoch 5/200
90/90 [==============================] - 0s 3ms/step - loss: 3.9494 - accuracy: 0.1585
Epoch 6/200
90/90 [==============================] - 0s 3ms/step - loss: 3.7049 - accuracy: 0.1719
Epoch 7/200
90/90 [==============================] - 0s 3ms/step - loss: 3.5596 - accuracy: 0.2143
Epoch 8/200
90/90 [==============================] - 0s 3ms/step - loss: 3.3569 - accuracy: 0.2321
Epoch 9/200
90/90 [==============================] - 0s 3ms/step - loss: 3.2161 - accuracy: 0.2478
Epoch 10/200
90/90 [==============================] - 0s 3ms/step - loss: 3.0905 - accuracy: 0.2746
Epoch 1

90/90 [==============================] - 0s 3ms/step - loss: 0.6574 - accuracy: 0.7790
Epoch 79/200
90/90 [==============================] - 0s 3ms/step - loss: 0.6679 - accuracy: 0.7746
Epoch 80/200
90/90 [==============================] - 0s 3ms/step - loss: 0.5895 - accuracy: 0.8237
Epoch 81/200
90/90 [==============================] - 0s 3ms/step - loss: 0.5945 - accuracy: 0.8304
Epoch 82/200
90/90 [==============================] - 0s 3ms/step - loss: 0.6409 - accuracy: 0.7835
Epoch 83/200
90/90 [==============================] - 0s 3ms/step - loss: 0.6566 - accuracy: 0.7946
Epoch 84/200
90/90 [==============================] - 0s 3ms/step - loss: 0.6742 - accuracy: 0.7879
Epoch 85/200
90/90 [==============================] - 0s 3ms/step - loss: 0.6030 - accuracy: 0.7991
Epoch 86/200
90/90 [==============================] - 0s 3ms/step - loss: 0.5683 - accuracy: 0.8058
Epoch 87/200
90/90 [==============================] - 0s 3ms/step - loss: 0.5643 - accuracy: 0.8103
Epoch 88/200


90/90 [==============================] - 0s 3ms/step - loss: 0.4157 - accuracy: 0.8504
Epoch 160/200
90/90 [==============================] - 0s 3ms/step - loss: 0.4252 - accuracy: 0.8304
Epoch 161/200
90/90 [==============================] - 0s 3ms/step - loss: 0.3644 - accuracy: 0.8705
Epoch 162/200
90/90 [==============================] - 0s 3ms/step - loss: 0.3952 - accuracy: 0.8549
Epoch 163/200
90/90 [==============================] - 0s 3ms/step - loss: 0.4224 - accuracy: 0.8504
Epoch 164/200
90/90 [==============================] - 0s 3ms/step - loss: 0.4359 - accuracy: 0.8549
Epoch 165/200
90/90 [==============================] - 0s 3ms/step - loss: 0.3572 - accuracy: 0.8661
Epoch 166/200
90/90 [==============================] - 0s 3ms/step - loss: 0.4032 - accuracy: 0.8549
Epoch 167/200
90/90 [==============================] - 0s 3ms/step - loss: 0.3840 - accuracy: 0.8594
Epoch 168/200
90/90 [==============================] - 0s 3ms/step - loss: 0.4117 - accuracy: 0.8415
Epoc

# FOR PREDICTING RESPONSE
### **You have to add or run this below script by loading model**

In [ ]:
from keras.models import load_model
import json
import pickle
import numpy as np
import nltk
import random

model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

lemmatizer = nltk.stem.WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word.lower() not in stop_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

start = True
while start:
    query = input('Enter Message:')
    if query in ['quit','exit','bye']:
        start = False
        continue
    try:
        res = chatbot_response(query)
        print(res)
    except:
        print('You may need to rephrase your question.')


Enter Message:mini
1/1 [==============================] - 0s 133ms/step
You may need to rephrase your question.
Enter Message:list of apprentices
1/1 [==============================] - 0s 14ms/step
The apprenticeship program at iServe Africa offers training opportunities for young individuals interested in Christian ministry. Contact the regional ministry staff for information on apprenticeship intake and opportunities.
Enter Message:some apprentices in the program 
1/1 [==============================] - 0s 23ms/step
The apprenticeship program at iServe Africa offers training opportunities for young individuals interested in Christian ministry. Contact the regional ministry staff for information on apprenticeship intake and opportunities.
Enter Message:current apprentices
1/1 [==============================] - 0s 31ms/step
- Samuel (second mention): Served at Buruburu Baptist Church in Nairobi county for the first year, continuing for the second year.
Enter Message:all current apprenti